### Module 5: SQL for Data Analysis

This module introduces SQL concepts commonly used by data analysts to query structured data. SQL queries are used to answer the same business questions explored with Pandas, reinforcing analytical thinking across tools.

The focus is on translating business requirements into efficient database queries and understanding how SQL complements Python-based analysis in real-world data workflows.

## Setting SQlite and loading data

In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv("../data/retail_sales_cleaned.csv")
df.head()

,order_id,order_date,region,product,category,quantity,unit_price
0,1001,2023-01-05,North,Laptop,Electronics,2.0,750.0
1,1002,2023-01-07,South,Mobile,Electronics,8.0,300.0
2,1003,2023-01-10,East,Chair,Furniture,10.0,45.0
3,1004,2023-01-15,West,Table,Furniture,8.0,120.0
4,1005,2023-01-20,Unknown,Headphones,Electronics,8.0,60.0


In [3]:
conn = sqlite3.connect("../data/retail_sales.db")

In [5]:
df.to_sql("sales", conn, if_exists = "replace", index = False)

6

## Basic SQL Queries

In [6]:
query = """ SELECT * FROM sales LIMIT 5; """
pd.read_sql(query, conn)

,order_id,order_date,region,product,category,quantity,unit_price,revenue
0,1001,2023-01-05,North,Laptop,Electronics,2.0,750.0,1500.0
1,1002,2023-01-07,South,Mobile,Electronics,8.0,300.0,2400.0
2,1003,2023-01-10,East,Chair,Furniture,10.0,45.0,450.0
3,1004,2023-01-15,West,Table,Furniture,8.0,120.0,960.0
4,1005,2023-01-20,Unknown,Headphones,Electronics,8.0,60.0,480.0


In [4]:
df["revenue"] = df["quantity"]*df["unit_price"]

In [18]:
df.head(3)

,order_id,order_date,region,product,category,quantity,unit_price,revenue
0,1001,2023-01-05,North,Laptop,Electronics,2.0,750.0,1500.0
1,1002,2023-01-07,South,Mobile,Electronics,8.0,300.0,2400.0
2,1003,2023-01-10,East,Chair,Furniture,10.0,45.0,450.0


In [7]:
query = """ SELECT product, revenue FROM sales LIMIT 5; """
pd.read_sql(query, conn)

,product,revenue
0,Laptop,1500.0
1,Mobile,2400.0
2,Chair,450.0
3,Table,960.0
4,Headphones,480.0


## Filtering with WHERE and GROUP BY with some Practice Questions

In [8]:
query = """ SELECT * FROM sales WHERE category = "Electronics"; """
pd.read_sql(query, conn)

,order_id,order_date,region,product,category,quantity,unit_price,revenue
0,1001,2023-01-05,North,Laptop,Electronics,2.0,750.0,1500.0
1,1002,2023-01-07,South,Mobile,Electronics,8.0,300.0,2400.0
2,1005,2023-01-20,Unknown,Headphones,Electronics,8.0,60.0,480.0


In [9]:
query = """ SELECT category, SUM("revenue") AS total_revenue FROM sales GROUP BY category; """
pd.read_sql(query, conn)

,category,total_revenue
0,Electronics,4380.0
1,Furniture,1500.0


In [10]:
query = """
SELECT category, SUM(revenue) AS total_revenue
FROM sales
GROUP BY category
ORDER BY total_revenue;
"""

pd.read_sql(query, conn)

,category,total_revenue
0,Furniture,1500.0
1,Electronics,4380.0


In [11]:
query = """ SELECT category, SUM(quantity) AS total_items_sold
FROM sales GROUP BY category HAVING total_items_sold > 5; """
pd.read_sql(query, conn)

,category,total_items_sold
0,Electronics,18.0
1,Furniture,19.0


In [12]:
query = """ SELECT region, AVG(unit_price) AS average_unit_price
FROM sales GROUP BY region HAVING average_unit_price > 100; """
pd.read_sql(query, conn)

,region,average_unit_price
0,North,750.0
1,South,195.0
2,West,120.0


In [13]:
query = """ SELECT product, SUM(revenue) AS total_revenue
FROM sales GROUP BY product HAVING total_revenue < 500; """
pd.read_sql(query, conn)

,product,total_revenue
0,Chair,450.0
1,Headphones,480.0
2,Sofa,90.0


## Creating New Table "product"

In [19]:
product_df = pd.DataFrame({ "product": ["Laptop", "Mobile", "Chair", "Table"],
                           "supplier": ["TechCorp", "PhoneInc", "FurniCo", "FurniCo"],
                           "warranty_years": [2, 1, None, None] })
product_df

,product,supplier,warranty_years
0,Laptop,TechCorp,2.0
1,Mobile,PhoneInc,1.0
2,Chair,FurniCo,NaN
3,Table,FurniCo,NaN


In [18]:
product_df.to_sql("products", conn, if_exists="replace", index = False)

4

In [51]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,sales
1,products


In [15]:
pd.read_sql("PRAGMA table_info(sales);", conn)
pd.read_sql("PRAGMA table_info(products);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,product,TEXT,0,None,0
1,1,supplier,TEXT,0,None,0
2,2,warranty_years,REAL,0,None,0


In [16]:
query = """SELECT * FROM sales;"""
pd.read_sql(query, conn)

,order_id,order_date,region,product,category,quantity,unit_price,revenue
0,1001,2023-01-05,North,Laptop,Electronics,2.0,750.0,1500.0
1,1002,2023-01-07,South,Mobile,Electronics,8.0,300.0,2400.0
2,1003,2023-01-10,East,Chair,Furniture,10.0,45.0,450.0
3,1004,2023-01-15,West,Table,Furniture,8.0,120.0,960.0
4,1005,2023-01-20,Unknown,Headphones,Electronics,8.0,60.0,480.0
5,1006,2023-01-22,South,Sofa,Furniture,1.0,90.0,90.0


In [17]:
query = """SELECT * FROM products;"""
pd.read_sql(query, conn)

,product,supplier,warranty_years
0,Laptop,TechCorp,2.0
1,Mobile,PhoneInc,1.0
2,Chair,FurniCo,NaN
3,Table,FurniCo,NaN


In [20]:
query = """SELECT s.product, p.supplier, s.revenue
FROM sales AS s
INNER JOIN products AS p
ON s.product = p.product;"""
pd.read_sql(query, conn)

,product,supplier,revenue
0,Laptop,TechCorp,1500.0
1,Mobile,PhoneInc,2400.0
2,Chair,FurniCo,450.0
3,Table,FurniCo,960.0


In [21]:
query = """SELECT s.product, p.supplier, s.revenue
FROM sales AS s
LEFT JOIN products AS p
ON s.product = p.product;"""
pd.read_sql(query, conn)

,product,supplier,revenue
0,Laptop,TechCorp,1500.0
1,Mobile,PhoneInc,2400.0
2,Chair,FurniCo,450.0
3,Table,FurniCo,960.0
4,Headphones,None,480.0
5,Sofa,None,90.0


In [22]:
query = """SELECT
    s.product,
    s.revenue
FROM sales AS s
LEFT JOIN products AS p
    ON s.product = p.product
WHERE p.product IS NULL;"""

pd.read_sql(query, conn)

,product,revenue
0,Headphones,480.0
1,Sofa,90.0


In [23]:
query = """SELECT SUM(s.revenue) AS total_revenue, p.supplier FROM sales AS s
LEFT JOIN products AS p ON s.product = p.product GROUP BY p.supplier ORDER BY total_revenue DESC;"""
pd.read_sql(query, conn)

,total_revenue,supplier
0,2400.0,PhoneInc
1,1500.0,TechCorp
2,1410.0,FurniCo
3,570.0,None


In [24]:
query = """SELECT p.supplier, COUNT(*) AS total_sales
FROM sales AS s LEFT JOIN products AS p ON s.product = p.product GROUP BY p.supplier;"""
pd.read_sql(query, conn)

,supplier,total_sales
0,None,2
1,FurniCo,2
2,PhoneInc,1
3,TechCorp,1


In [25]:
query = """SELECT AVG(s.revenue) AS average_revenue, p.warranty_years 
FROM sales AS s LEFT JOIN products AS p ON s.product = p.product
GROUP BY p.warranty_years ORDER BY average_revenue DESC;"""
pd.read_sql(query, conn)

,average_revenue,warranty_years
0,2400.0,1.0
1,1500.0,2.0
2,495.0,NaN


In [27]:
query = """SELECT p.supplier, SUM(s.revenue) AS total_revenue
FROM sales AS s LEFT JOIN products AS p
ON s.product = p.product
GROUP BY p.supplier
HAVING SUM(s.revenue) > 1000
ORDER BY total_revenue DESC;"""
pd.read_sql(query, conn)

,supplier,total_revenue
0,PhoneInc,2400.0
1,TechCorp,1500.0
2,FurniCo,1410.0
